In [1]:
import random
random.seed(42)

from collections import namedtuple

import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D

import numpy as np

from jim import Vector, with_state, Universe, exhaust

In [2]:
class AttractorUniverse(Universe):
    def __init__(self, a, b, c):
        super().__init__(Vector.null_vector(3))
        self.params = a, b, c
    
    def add(self, body):
        super().add(body)
        body.attractor = self

In [3]:
@with_state
class LorenzBody:
    State = namedtuple("LorenzBodyState", "r, v")
    
    def __init__(self, r):
        self.r = r
        self.attractor = None
        
    def move(self, dt):
        self.r += self.v * dt
        
    @property
    def v(self):
        a, b, c = self.attractor.params
        x, y, z = self.r
        return Vector(
            a * (y - x),
            x * (b - z) - y,
            x * y - c * z
        )

In [4]:
n_bodies = 8

In [5]:
bodies = [
    LorenzBody(Vector.random(3, limit=20))
    for _ in range(n_bodies)
]

In [6]:
attractor = AttractorUniverse(10, 28, 8/3)
attractor.add_all(bodies)

----

In [7]:
dt = 1. / 100
frames = 1800

In [8]:
states = list(attractor.simulate_shortstep(frames * dt + dt, dt))

In [9]:
fig = plt.figure(figsize=(12.80, 7.20))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
ax = fig.add_subplot(
    111,
    aspect="equal",
    autoscale_on=False,
    projection="3d",
)
ax.grid()

axlim = 50

ax.set_xlim3d(-axlim, axlim)
ax.set_ylim3d(-axlim, axlim)
ax.set_zlim3d(-axlim, axlim)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

Ein roter Punkt im Ursprung zur Orientierung:

In [10]:
ax.scatter([0], [0], c="red");

In [11]:
particles, = ax.plot([], [], 'o', ms=2)
traces = [
    ax.plot([], [], "-")[0]
    for _ in range(n_bodies)
]

In [12]:
def init():
    particles.set_data([], [])
    particles.set_3d_properties([])
    for trace in traces:
        trace.set_data([], [])
        trace.set_3d_properties([])
    
    return (particles, *traces)

In [13]:
def animate(i):
    state = states[i]
    
    particles.set_data(
        [s.r[0] for s in state],
        [s.r[1] for s in state]
    )
    particles.set_3d_properties(
        [s.r[2] for s in state]
    )
    
    for n, trace in enumerate(traces):
        trace.set_data(
            [state[n].r[0] for state in states[:i + 1]],
            [state[n].r[1] for state in states[:i + 1]]
        )
        trace.set_3d_properties(
            [state[n].r[2] for state in states[:i + 1]]
        )
    
    return (particles, *traces)

In [14]:
def animate_with_rotation(i):
    ret = animate(i)
    ax.view_init(i * 360 / 900 - 90, i)
    return ret

In [15]:
ani = animation.FuncAnimation(
    fig,
    animate,
    frames=frames,
    interval=30,
    blit=False,
    init_func=init
)

In [16]:
plt.show()

In [17]:
# save the animation as an mp4.
#ani.save('lorenz_attractor_rotating.mp4', fps=30, extra_args=['-vcodec', 'libx264'])